In [1]:
%load_ext sql
%sql postgresql://appdev@data:5432/appdev

'Connected: appdev@appdev'

# In the first part you'll be processing the data in the geonames table.

## Find the tuple with the largest population in geoname

In [20]:
%%sql 
DROP VIEW IF EXISTS total_pop;
CREATE VIEW total_pop AS
SELECT * FROM geonames ORDER BY population DESC LIMIT 1;

SELECT * FROM geonames ORDER BY population DESC LIMIT 1;

Done.
Done.
1 rows affected.


geonameid,name,longitude,latitude,feature_class,feature_code,country_code,admin1_code,admin2_code,population,elevation,timezone
305104,Levant,36.0,34.0,L,RGN,None,None,None,44990000,None,Asia/Beirut


## Now calculate the distance from all locations to the tuple above using the <-> function

In [21]:
%%sql
SELECT point(longitude, latitude) <-> (SELECT point(longitude, latitude) FROM total_pop) AS distance 
FROM geonames LIMIT 5;

5 rows affected.


distance
35.5187260391022
35.5403912827645
35.4976906065648
35.5211340049413
35.5230848613757


## Now calculate the distance from all locations to the tuple above using the haversine distance

In [23]:
%%sql CREATE OR REPLACE FUNCTION public.geodistance(alat double precision, alng double precision, blat double precision, blng double precision)
  RETURNS double precision AS
$BODY$
SELECT asin(
  sqrt(
    sin(radians($3-$1)/2)^2 +
    sin(radians($4-$2)/2)^2 *
    cos(radians($1)) *
    cos(radians($3))
  )
) * 7926.3352 AS distance;
$BODY$
  LANGUAGE sql IMMUTABLE
  COST 100;

Done.


[]

In [30]:
%%sql SELECT geodistance(latitude, longitude,
                        (SELECT latitude FROM total_pop),
                        (SELECT longitude FROM total_pop)) 
FROM geonames LIMIT 5;

5 rows affected.


geodistance
1948.31948131202
1948.97544410187
1946.67835552112
1948.51401122396
1948.68370586011


## Plot the latitude and longitude of the largest 1000 populations on a 2-D scatter plot

In [31]:
import pandas as pd
import matplotlib.pyplot as plt